## 1. Prepare data

### 1.1 Generate simulated data

In [1]:
using TorusEvol
using Distributions

# Underlying evolutionary process
t = 0.3; λ=0.03; μ=0.031; r=0.4
τ = TKF92([t], λ, μ, r)
S = WAG_SubstitutionProcess()
μ_𝜙=-2.0; μ_𝜓=2.35; σ_𝜙=0.4; σ_𝜓=0.4; α_𝜙=0.5; α_𝜓=1.0; α_cov=0.1; γ=0.2
Θ = JumpingWrappedDiffusion(μ_𝜙, μ_𝜓, σ_𝜙, σ_𝜓, α_𝜙, α_𝜓, α_cov, γ)
ξ = MixtureProductProcess([1.0], hcat([S, Θ]))
pair_chain_dist = ChainJointDistribution(ξ, τ)

# Generate data
simulated_data = Tuple{ObservedChain, ObservedChain}[]
n = 3 # data size
for i ∈ 1:n
    push!(simulated_data, rand(pair_chain_dist))
end

# Render a random sample from the data
(A, B) = rand(simulated_data)
chainA = from_primary_dihedrals(Int.(data(A)[1]), data(A)[2])
chainB = from_primary_dihedrals(Int.(data(B)[1]), data(B)[2])
lp = logpdf(pair_chain_dist, (A, B))
print("The log pdf of A and B is $lp")
render(chainA, chainB; aligned=true)

[ Info: Precompiling TorusEvol [4b860a26-b3bc-4b38-a9ed-83c1dc5d19b0]
ERROR: LoadError: syntax: unexpected ")"
Stacktrace:
 [1] top-level scope
   @ C:\Users\stefa\Dropbox\Jotun Hein - Part C Projects\Manolache\Code\TorusEvol\src\distributions\AlignmentProcess.jl:96
 [2] include(mod::Module, _path::String)
   @ Base .\Base.jl:457
 [3] include(x::String)
   @ TorusEvol C:\Users\stefa\Dropbox\Jotun Hein - Part C Projects\Manolache\Code\TorusEvol\src\TorusEvol.jl:1
 [4] top-level scope
   @ C:\Users\stefa\Dropbox\Jotun Hein - Part C Projects\Manolache\Code\TorusEvol\src\TorusEvol.jl:151
 [5] include
   @ .\Base.jl:457 [inlined]
 [6] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt128}}, source::Nothing)
   @ Base .\loading.jl:2012
 [7] top-level scope
   @ stdin:2
in expression starting at C:\Users\stefa\Dropbox\Jotun Hein - Part C Projects\Manolache\C

LoadError: Failed to precompile TorusEvol [4b860a26-b3bc-4b38-a9ed-83c1dc5d19b0] to "D:\\Programs\\julia_depot\\compiled\\v1.9\\TorusEvol\\jl_1E08.tmp".

### 1.2 Load real data

## 2. Parameter Inference Bayesian Model

### 2.1 Set up priors for evolutionary processes

In [2]:
using Turing, DynamicPPL
using LinearAlgebra
using LogExpFunctions 
using Plots, StatsPlots
using Random

import Base: length, eltype
import Distributions: _rand!, logpdf

Turing.setprogress!(true)

struct ScaledBeta <: ContinuousUnivariateDistribution 
    be::Beta 
    function ScaledBeta(α::Real, β::Real)
        new(Beta(α, β))
    end
end
Distributions.rand(rng::AbstractRNG, d::ScaledBeta) = rand(d.be)*2 - 1
Distributions.logpdf(d::ScaledBeta, x::Real) = logpdf(d.be, (x+1) / 2)


struct CompetingExponential <: ContinuousMultivariateDistribution 
    ex::Exponential
    function CompetingExponential(rate::Real)
        new(Exponential(rate))
    end
end 
Base.eltype(d::CompetingExponential) = Float64 
Base.length(d::CompetingExponential) = 2

function Distributions._rand!(rng::AbstractRNG, d::CompetingExponential, x::AbstractVector{<:Real})
    λ = rand(rng, d.ex)
    μ = rand(rng, d.ex)
    if λ > μ 
        tmp = λ; λ = μ; μ=tmp 
    end
    x .= [λ, μ]
    return x
end

function Distributions._logpdf(d::CompetingExponential, x::AbstractArray)
    if x[1] > x[2]
        return -Inf
    end
    return log(2) + logpdf(d.ex, x[1]) + logpdf(d.ex, x[2])
end

@model function tkf92_prior()
    λμ ~ CompetingExponential(1.0)
    λ = λμ[1]; μ = λμ[2]
    r ~ Uniform(0.0, 1.0)

    # Require birth rate lower than death rate
    if λ > μ || λ ≤ 0 || μ ≤ 0 || r ≤ 0 || r ≥ 1
        μ = NaN; λ = NaN
    end
    return λ, μ, r
end;

@model function jwndiff_prior()
    μ ~ filldist(Uniform(-π, π), 2)
    σ² ~ filldist(Gamma(π * 0.1), 2)
    α ~ filldist(Gamma(π * 0.1), 2)
    γ ~ Exponential(1.0)   # jumping rate
    α_corr ~ ScaledBeta(3, 3)
    
    # Require valid covariance matrices
    if any(σ² .≤ 0) || any(α .≤ 0) || γ ≤ 0 
        σ² .= NaN; α .= NaN; γ = NaN
    end
    α_cov = α_corr * sqrt(α[1] * α[2])
    if α_cov^2 > α[1]*α[2]
         α_cov = NaN
    end
    
    return μ[1], μ[2], sqrt(σ²[1]), sqrt(σ²[2]), α[1], α[2], α_cov, γ
end;

[ Info: [Turing]: progress logging is enabled globally
[ Info: [AdvancedVI]: global PROGRESS is set as true


### 2.2 Set up sampler

In [3]:
torus_proposal(v) = MixtureModel([WrappedNormal(v, I), WrappedNormal(v, 20*I)], [0.8, 0.2])
mv_rw_proposal(v::AbstractVector, cov) = MvNormal(v, cov)
rw_proposal(x, var) = Normal(x, var)


sampler = Gibbs(MH(:t => v -> rw_proposal(v, 0.2)),
                MH(Symbol("Θ.μ") => v -> torus_proposal(v)),
                MH(Symbol("Θ.σ²") => v -> mv_rw_proposal(v, 0.4*I)),
                MH(Symbol("Θ.α") => v -> mv_rw_proposal(v, 0.4*I)),
                MH(Symbol("Θ.α_corr") => x -> rw_proposal(x, 0.5)),
                MH(Symbol("Θ.γ") => x -> rw_proposal(x, 0.5)),
                MH(Symbol("τ.λμ") => v -> mv_rw_proposal(v, [0.4 0.1; 0.1 0.6])),
                MH(Symbol("τ.r") => x -> rw_proposal(x, 0.5))
               );

In [4]:
using Memoization 

@memoize get_αs(pairs) = get_α.(Ref(TKF92([1.0], 0.2, 0.3, 0.4)), pairs)
@memoize get_Bs(pairs) = get_B.(pairs)

get_Bs (generic function with 1 method)

### 2.3 Prepare probabilistic model

In [5]:
@model function pair_param_inference_simple(pairs)
    # ____________________________________________________________________________________________________
    # Step 1 - Sample prior parameters
    
    # Time parameter
    t ~ Exponential(1.0) 
    # Alignment parameters
    @submodel prefix="τ" Λ = tkf92_prior()
    # Dihedral parameters 
    @submodel prefix="Θ" Ξ = jwndiff_prior()
    # Check parameter validity 
    if t ≤ 0 || any(isnan.(Ξ)) || any(isnan.(Λ))
        Turing.@addlogprob! -Inf; return
    end
    
    # ____________________________________________________________________________________________________
    # Step 2 - Construct processes 
    
    # Substitution Process - no parameters for simplicity, use fully empirical model
    S = WAG_SubstitutionProcess()
    # Dihedral Process
    Θ = JumpingWrappedDiffusion(Ξ...)
    # Joint sequence-structure site level process with one regime
    ξ = MixtureProductProcess([1.0], hcat([S, Θ]))
    
    # Alignment model
    τ = TKF92([t], Λ...)
    
    # Chain level model
    Γ = ChainJointDistribution(ξ, τ)
    
    # ____________________________________________________________________________________________________
    # Step 3 - Observe each pair X, Y by proxy of their joint probability, marginalising over alignments
    α = get_αs(pairs)
    Threads.@threads for i ∈ eachindex(pairs)
        X, Y = pairs[i]
        # (X, Y) ~ ChainJointDistribution(ξ, τ)
        Turing.@addlogprob! logpdfα!(α[i], Γ, (X, Y))
    end
        
    return Γ
end;

### 2.4 Sample from the model and check results

In [6]:
num_samples = 300
num_chains = 3
model = pair_param_inference_simple(simulated_data)

LoadError: UndefVarError: `simulated_data` not defined

In [7]:
chain = sample(model, sampler, MCMCThreads(), num_samples, num_chains)
p = plot(chain, fontfamily="JuliaMono")

LoadError: UndefVarError: `model` not defined